파이썬의 동적 특성으로 인해 실행 시간 성능이 예상과 달라 놀랄 떄가 있다. 

느릴 것으로 예상한 연산이 실제로는 아주 빠르거나 **(예: 문자열 조작, 제너레이터)**

빠를 것으로 예산한 연산이 실제로는 아주 느리거나 **(예: 애트리뷰트 접근, 함수 호출)**

그러므로 가장 좋은 접근 방법은 직관을 무시하고 직접 프로그램의 성능을 측정하는 것이다..


In [2]:
def insertion_sort(data):
    result = []
    for value in data:
        insertion_value(result, value)
        
    return result

def insertion_value(array, value):
    for i, existing in enumerate(array):
        if existing > value:
            array.insert(i, value) # array가 깊은 복사를 하는 것이아니라, 얕은 복사를 하므로 여기에 넣어도 기존 result에 잘 들어간다..
            return
    
    array.append(value)

In [4]:
from random import randint

max_size = 10**4

data = [randint(0, max_size) for _ in range(max_size)]

test = lambda: insertion_sort(data)

파이썬에는 두 가지 내장 프로파일러가 있다.

1. 순수 파이썬 (profile)

2. C 확장 모듈인 (cProfile)

cProfile이 더 낫다. 프로파일 대상 프로그램의 성능에 최소로 영향을 미치기 때문ㅇ 

순수 파이썬은 부가 비용이 너무 많이 들어서 결과가 왜곡 될 수 있다.

In [6]:
from cProfile import Profile

profiler = Profile()
profiler.runcall(test)

from pstats import Stats

stats = Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative') # 누적 통계
stats.print_stats()

         20003 function calls in 5.970 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.970    5.970 296762891.py:7(<lambda>)
        1    0.006    0.006    5.970    5.970 1437399145.py:1(insertion_sort)
    10000    5.951    0.001    5.964    0.001 1437399145.py:8(insertion_value)
     9984    0.013    0.000    0.013    0.000 {method 'insert' of 'list' objects}
       16    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




- ncalls : 프로파일링 기간동안 함수가 몇번 호출됬는가?

- tottime : 프로파일링 기간 동안 대상함수를 실행하는 데 걸린 시간의 합계 
            대상 함수가 다른 함수를 호출한 경우, 이 다른 함수를 실행하는데 걸리는 시간은 제외된다.

- tottime percall : 함수가 호출될 때마다 걸린 시간의 평균을 보여준다. 이역시 다른 함수를 실행하기 위한 평균 시간은 제외된다
                    이 값은 tottime을 ncalls로 나눈 값과 같다

- cumtime : 함수를 실행하는 데 누적 걸린 시간을 보여준다. 이 시간에는 대상 함수가 호출한 다른 함수를 실행하는데 걸린 시간이 모두 포함된다.

- cumtime percall : 프로파일링 기간동안 함수가 호출될 때마다 걸린 누적 시간의 평균, 대상 함수가 호출한 다른 함수를 실행하는데 걸린 시간이 모두 포함됨

우리 코드에서 누적시간으로 CPU를 가장 많이 사용한 함수는 insert_value 함수라는 점을 알 수 있다. 이 함수를 bisect 내장 모듈을 통해 다시 구현해본다

In [7]:
from bisect import bisect_left

def insert_value(array, value):
    i = bisect_left(array, value)
    array.insert(i, value)

In [13]:
def insertion_sort(data):
    result = []
    for value in data:
        insert_value(result, value) # 
        
    return result

from random import randint

max_size = 10**4

data = [randint(0, max_size) for _ in range(max_size)]

# print(data)

test = lambda: insertion_sort(data)

from cProfile import Profile

profiler = Profile()
profiler.runcall(test)

from pstats import Stats

stats = Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative') # 누적 통계
stats.print_stats()

[1705, 2175, 1914, 2907, 268, 4058, 3143, 8630, 2086, 5374, 6915, 1985, 1552, 3188, 9557, 16, 5459, 5093, 1046, 1852, 5417, 2157, 603, 7321, 3390, 4178, 6184, 5145, 7315, 926, 6693, 5084, 3837, 4448, 7685, 4218, 5594, 8884, 5924, 4929, 316, 7913, 9175, 3657, 765, 6224, 2387, 7798, 7669, 6735, 9040, 7811, 2831, 5987, 7530, 4197, 5659, 8923, 5098, 8362, 2219, 241, 5698, 762, 9319, 5150, 5862, 8993, 8609, 6442, 9383, 5421, 5281, 2679, 9830, 9237, 2757, 2784, 1277, 9596, 8334, 7722, 4982, 7024, 2108, 5699, 5619, 6035, 5000, 7786, 3948, 2799, 5045, 4761, 8543, 2975, 6605, 9133, 7830, 123, 4314, 5873, 5747, 2466, 424, 7277, 3443, 3415, 9710, 7526, 2798, 979, 4924, 623, 1014, 7866, 9318, 3681, 5611, 712, 4447, 8562, 8997, 9109, 4400, 3265, 3282, 2870, 9992, 1035, 2867, 8093, 6947, 7057, 8227, 2978, 9303, 5035, 2854, 7310, 6642, 9119, 3217, 7999, 2276, 2685, 5663, 4621, 438, 9832, 7986, 8230, 2057, 2983, 8550, 9002, 4201, 8544, 9955, 7798, 6877, 9083, 7358, 4408, 3344, 2988, 4195, 9382, 5309, 

전체 프로그램을 프로파일링 했는데 공통 유틸리티 함수가 대부분의 실행시간을 차지한다는 사실을 발견할 때도 있다.

In [14]:
def my_utility(a, b):
    c = 1
    for i in range(100):
        c += a * b
        
def first_func():
    for _ in range(1000):
        my_utility(4, 5)
    
def second_func():
    for _ in range(10):
        my_utility(1, 3)
        
def my_program():
    for _ in range(20):
        first_func()
        second_func()
        
profiler = Profile()
profiler.runcall(my_program)

stats = Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative') # 누적 통계
stats.print_stats()

         20242 function calls in 0.518 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.518    0.518 1551736271.py:14(my_program)
       20    0.012    0.001    0.513    0.026 1551736271.py:6(first_func)
    20200    0.505    0.000    0.505    0.000 1551736271.py:1(my_utility)
       20    0.000    0.000    0.005    0.000 1551736271.py:10(second_func)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




myutility 함수가 왜 이렇게 많이 호출됬는지 즉시 알아보기는 어렵다

프로그램 코드를 살펴보면 my_utility 호출하는 부분이 여러 곳이라는 사실을 알 수 있지만 여전히 혼란스럽다.

각 함수를 프로파일링 한 정보에 대해 그 함수를 호출한 함수들이 얼마나 기여했는지를 보여주는 print_callers 메서드를 제공



왼쪽에 호출된 함수를, 오른쪽에 그 함수를 호출한 함수를 보여준다.

first_func가 my_utility 함수를 가장 많이 썼다는 점을 알 수 있다.

In [15]:
stats.print_callers()

   Ordered by: cumulative time

Function                                          was called by...
                                                      ncalls  tottime  cumtime
1551736271.py:14(my_program)                      <- 
1551736271.py:6(first_func)                       <-      20    0.012    0.513  1551736271.py:14(my_program)
1551736271.py:1(my_utility)                       <-   20000    0.501    0.501  1551736271.py:6(first_func)
                                                         200    0.004    0.004  1551736271.py:10(second_func)
1551736271.py:10(second_func)                     <-      20    0.000    0.005  1551736271.py:14(my_program)
{method 'disable' of '_lsprof.Profiler' objects}  <- 


